In [1]:
import aaai20
import numpy as np
import pandas as pd
from aaai20.io import build_filesystem, filename_query, filename_dataset, filename_model
from joblib import dump, load

from mercs.utils.encoding import code_to_query, encode_attribute, get_att, query_to_code

In [2]:
import mercs

from mercs import Mercs

In [3]:
nb_atts = 10**4

a = np.zeros((10**4, nb_atts))

idxs = np.random.randint(nb_atts, size=4500)

In [4]:
t = a.take(1, axis=1)

t.size

10000

In [5]:
# Get queries

fs = build_filesystem()

dataset='ad'

In [6]:
fn_qry = filename_query(dataset, suffix="default")
q_codes = np.load(fn_qry)

In [7]:
fn_train = filename_dataset(dataset, step=2, suffix="train", extension="csv")
df_train = pd.read_csv(fn_train, header=None, index_col=None)
train = df_train.values

fn_test = filename_dataset(dataset, step=2, suffix="test", extension="csv")
df = pd.read_csv(fn_test, header=None, index_col=None)

In [20]:
clf = Mercs(max_depth=2, nb_iterations=2, n_jobs=8, selection_algorithm="random", nb_targets=2, fraction_missing=[0.3,0.5], verbose=1)
clf.fit(train, verbose=1)


/cw/dtailocal/repos/mercs/src/mercs/algo/induction.py:99: UserWarning: 
        Training is being parallellized using Joblib. Number of jobs = 8
        
  warnings.warn(msg)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 688 tasks      | elapsed:   12.4s
[Parallel(n_jobs=8)]: Done 1688 tasks      | elapsed:   28.1s
[Parallel(n_jobs=8)]: Done 2768 out of 2768 | elapsed:   41.3s finished


In [9]:
len(clf.m_list)

2768

In [10]:
q_idx = 16
q_code = q_codes[q_idx]

In [11]:
test = df.values
test = test.astype(float)
target_ids = get_att(q_code, kind="targ").tolist()

y_true = test[:, target_ids].copy()  # Extract ground truth
test[
    :, target_ids
] = np.nan  # Ensure the answers never enter the algorithm

In [25]:
%%prun
y_pred = clf.predict(test, q_code=q_code, prediction_algorithm='it', max_steps=8)

         16238619 function calls (16233827 primitive calls) in 8.655 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.137    3.137    4.130    4.130 digraph.py:643(add_edges_from)
        1    1.286    1.286    1.286    1.286 digraph.py:540(remove_nodes_from)
        1    1.273    1.273    6.852    6.852 q_diagram.py:38(build_diagram_single_layer)
  3818086    1.066    0.000    1.066    0.000 q_diagram.py:70(v_name)
  1917335    0.502    0.000    0.502    0.000 {method 'get' of 'dict' objects}
  3819343    0.394    0.000    0.396    0.000 {method 'update' of 'dict' objects}
  1905723    0.198    0.000    0.198    0.000 q_diagram.py:45(<lambda>)
      507    0.198    0.000    0.198    0.000 {method 'take' of 'numpy.ndarray' objects}
  1910372    0.102    0.000    0.102    0.000 {built-in method builtins.len}
  1909067    0.096    0.000    0.096    0.000 {method 'append' of 'list' objects}
   764869    0.088    

In [26]:
clf.model_data

{'ind_time': 50.46570422100194,
 'prd_time': 0.03580538799724309,
 'dia_time': 8.2302644819938,
 'infalgo_time': 0.014037425004062243,
 'dsk_time': 0.37453580900182715,
 'inf_time': 8.6546458360026}

In [23]:
%%prun
y_pred = clf.predict(test, q_code=q_code, prediction_algorithm='rw', max_steps=8, nb_walks=30)
y_pred

         9667336 function calls (9389834 primitive calls) in 9.169 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    18196    1.232    0.000    1.232    0.000 {method 'take' of 'numpy.ndarray' objects}
      720    0.932    0.001    2.141    0.003 vector_prediction.py:227(criterion)
   101969    0.651    0.000    0.651    0.000 {method 'reduce' of 'numpy.ufunc' objects}
206796/154042    0.523    0.000    1.752    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      241    0.336    0.001    0.416    0.002 digraph.py:643(add_edges_from)
    24804    0.323    0.000    3.126    0.000 _base.py:368(transform)
   448166    0.256    0.000    0.256    0.000 q_diagram.py:70(v_name)
      268    0.244    0.001    0.244    0.001 inference_v3.py:225(<dictcomp>)
   564427    0.229    0.000    0.305    0.000 graph.py:663(nodes)
 222693/1    0.226    0.000    4.973    4.973 inference_v3.py:159(compute)
    

In [24]:
clf.model_data

{'ind_time': 50.46570422100194,
 'prd_time': 2.4462241689980146,
 'dia_time': 1.078854504004994,
 'infalgo_time': 0.6816971070002182,
 'dsk_time': 4.974302597001952,
 'inf_time': 9.181080408998241}